In [1]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la
import scipy.integrate as integ
import ipywidgets as widgets

# Pomocnicze materiały do prędkości i przyspieszenia

Uwaga: ponieważ część nazw zmiennych użytych w przykładach pokrywa się, każdą komórkę należy wykonać przed użyciem interaktywnych wykresów.

# Prędkość chwilowa

Poniższy przykład obrazuje koncepcję prędkości chwilowej.

In [2]:
n = 5
t = np.linspace(0, 10, 100)
x = np.sin(t) + t**3 / 1000
v = np.cos(t) + 3 * t**2 / 1000


def xfun(n):
    tn = np.linspace(0, 10, n + 1)
    xn = np.sin(tn) + tn**3 / 1000
    vn = (np.roll(xn, -1) - xn) / (np.roll(tn, -1) - tn)
    vn[-1] = vn[-2]

    plt.figure(figsize = (20, 20))
    
    plt.subplot(211)
    plt.plot(t, x)
    plt.plot(tn, xn, 'r-')
    plt.xlabel('t')
    plt.ylabel('x')
    plt.grid(linestyle = '--')
    
    plt.subplot(212)
    plt.plot(t, v)
    plt.step(tn, vn, 'r-', where = 'post')
    plt.xlabel('t')
    plt.ylabel('v')
    plt.grid(linestyle = '--')    
    
widgets.interact(xfun, n = widgets.IntSlider(min=2, max=100, step=1, value=2))

interactive(children=(IntSlider(value=2, description='n', min=2), Output()), _dom_classes=('widget-interact',)…

<function __main__.xfun(n)>

# Przyspieszenia styczne i normalne - rzut ukośny

Przyspieszenia styczne i normalne dla rzutu ukośnego. Styczne wyraża się wzorem

$$a_s = \frac{dv}{dt},$$

gdzie $v$ to wartość (długość) wektora prędkości. Przyspieszenie normalne można natomiast wyznaczyć korzystając z zależnośći

$$a_n^2 = a^2 - a_s^2,$$

gdzie $a$ to wartość przyspieszenia całkowitego.

In [2]:
t, dt = np.linspace(0, 2, 101, retstep = True)
g = 9.81

def field(y, t):
    x, vx, y, vy = y
    return vx, 0, vy, -g

res = integ.odeint(field, (0, 10, 0, 10), t)
x, vx, y, vy = res[:, 0], res[:, 1], res[:, 2], res[:, 3]

def plot(i, scale):
    fig, ax = plt.subplots()
    fig.set_size_inches((20, 10))
    ax.set_aspect(1)
    ax.grid(linestyle = '--')
    ax.plot(x, y, linewidth = 2)
    ax.set_ylim((0, 10))
    ax.set_xlim((0, 20))
    
    v = np.array([vx[i], vy[i]])
    v /= la.norm(v)
    
    vv = np.array([vx[i + 1], vy[i + 1]])
    vv /= la.norm(vv)
    
    nrm = vv - v
    nrm /= la.norm(nrm)
    
    if scale:
        a = np.array([0, -g])
        a *= (3 / g)
        atan = np.dot(a, v)
        anor = np.dot(a, nrm)
        v *= atan
        nrm *= anor
        ax.arrow(x[i], y[i], 0, -3, width = 0.15, color = 'blue', zorder = 2)
    
    ax.arrow(x[i], y[i], v[0], v[1], width = 0.15, color = 'red', zorder = 2)
    ax.arrow(x[i], y[i], nrm[0], nrm[1], width = 0.15, color = 'red', zorder = 2)
    ax.plot([x[i]], [y[i]], 'bo', zorder = 3, ms = 10)
    
widgets.interact(plot, i = widgets.IntSlider(min=0, max=99, step=1, value=0),
                scale = widgets.Checkbox(value = False, description = "Skalowanie"))

interactive(children=(IntSlider(value=0, description='i', max=99), Checkbox(value=False, description='Skalowan…

<function __main__.plot(i, scale)>

# Przyspieszenia radialne i transwersalne - rzut ukośny

Przyspieszenie radialne wyraża się wzorem

$$a_r = \ddot{r} - r\dot{\phi}^2.$$

Przyspieszenie transwersalne można natomiast wyznaczyć korzystając z zależnośći

$$a_\phi = 2\dot{r}\dot{\phi} + r\ddot{\phi}.$$

In [3]:
t, dt = np.linspace(0, 2, 101, retstep = True)
g = 9.81

def field(y, t):
    x, vx, y, vy = y
    return vx, 0, vy, -g

res = integ.odeint(field, (0, 10, 0, 10), t)
x, vx, y, vy = res[:, 0], res[:, 1], res[:, 2], res[:, 3]

def plot(i, scale):
    fig, ax = plt.subplots()
    fig.set_size_inches((20, 10))
    ax.set_aspect(1)
    ax.grid(linestyle = '--')
    ax.plot(x, y, linewidth = 2)
    ax.set_ylim((0, 10))
    ax.set_xlim((0, 20))
    
    r = np.array([x[i], y[i]])
    r /= la.norm(r)
    
    rr = np.array([x[i + 1], y[i + 1]])
    rr /= la.norm(rr)
    
    phi = rr - r
    phi /= la.norm(phi)
    phi *= -1
    
    if scale:
        a = np.array([0, -g])
        a *= (3 / g)
        ar = np.dot(a, r)
        aphi = np.dot(a, phi)
        r *= ar
        phi *= aphi
        ax.arrow(x[i], y[i], 0, -3, width = 0.15, color = 'blue', zorder = 2)
    
    ax.arrow(x[i], y[i], r[0], r[1], width = 0.15, color = 'orange', zorder = 2)
    ax.arrow(x[i], y[i], phi[0], phi[1], width = 0.15, color = 'orange', zorder = 2)
    ax.plot([x[i]], [y[i]], 'bo', zorder = 3, ms = 10)
    ax.plot([0, x[i]], [0, y[i]], 'o--')
    
widgets.interact(plot, i = widgets.IntSlider(min=1, max=99, step=1, value=0),
                scale = widgets.Checkbox(value = False, description = "Skalowanie"))

interactive(children=(IntSlider(value=1, description='i', max=99, min=1), Checkbox(value=False, description='S…

<function __main__.plot(i, scale)>

W kodzie przyspieszenia zostały wyznaczone geometrycznie, poprzez rzuty wektora przyspieszenia na odpowiednie kierunki.

# Przyspieszenia radialne/transwersalne oraz styczne/normalne - porównanie

In [4]:
t, dt = np.linspace(0, 2, 101, retstep = True)
g = 9.81

def field(y, t):
    x, vx, y, vy = y
    return vx, 0, vy, -g

res = integ.odeint(field, (0, 10, 0, 10), t)
x, vx, y, vy = res[:, 0], res[:, 1], res[:, 2], res[:, 3]

def plot(i, scale):
    fig, ax = plt.subplots()
    fig.set_size_inches((20, 10))
    ax.set_aspect(1)
    ax.grid(linestyle = '--')
    ax.plot(x, y, linewidth = 2)
    ax.set_ylim((0, 10))
    ax.set_xlim((0, 20))
    
    r = np.array([x[i], y[i]])
    r /= la.norm(r)
    
    rr = np.array([x[i + 1], y[i + 1]])
    rr /= la.norm(rr)
    
    phi = rr - r
    phi /= la.norm(phi)
    phi *= -1
    
    if scale:
        a = np.array([0, -g])
        a *= (3 / g)
        ar = np.dot(a, r)
        aphi = np.dot(a, phi)
        r *= ar
        phi *= aphi
        ax.arrow(x[i], y[i], 0, -3, width = 0.15, color = 'blue', zorder = 2)
    
    ax.arrow(x[i], y[i], r[0], r[1], width = 0.15, color = 'orange', zorder = 2)
    ax.arrow(x[i], y[i], phi[0], phi[1], width = 0.15, color = 'orange', zorder = 2)
    ax.plot([x[i]], [y[i]], 'bo', zorder = 3, ms = 10)
    ax.plot([0, x[i]], [0, y[i]], 'o--')
    
    v = np.array([vx[i], vy[i]])
    v /= la.norm(v)
    
    vv = np.array([vx[i + 1], vy[i + 1]])
    vv /= la.norm(vv)
    
    nrm = vv - v
    nrm /= la.norm(nrm)
    
    if scale:
        a = np.array([0, -g])
        a *= (3 / g)
        atan = np.dot(a, v)
        anor = np.dot(a, nrm)
        v *= atan
        nrm *= anor
    
    ax.arrow(x[i], y[i], v[0], v[1], width = 0.15, color = 'red', zorder = 2)
    ax.arrow(x[i], y[i], nrm[0], nrm[1], width = 0.15, color = 'red', zorder = 2)
    ax.plot([x[i]], [y[i]], 'bo', zorder = 3, ms = 10)
    

widgets.interact(plot, i = widgets.IntSlider(min=1, max=99, step=1, value=0),
                scale = widgets.Checkbox(value = False, description = "Skalowanie"))

interactive(children=(IntSlider(value=1, description='i', max=99, min=1), Checkbox(value=False, description='S…

<function __main__.plot(i, scale)>